In [ ]:
def whatsAppContact(number):
    number = str(number)
    splited = [
        "".join(filter(str.isdigit, i))
        for i in number.replace("-", "").replace(")", " ").split(" ")
        if i
    ]
    if len(splited) == 2:
        if len(splited[0]) not in [2, 3]:
            out = False
        elif len(splited[1]) not in [8, 9]:
            out = False
        else:
            splited[0] = splited[0].zfill(3)
            splited[1] = splited[1].zfill(9)
            out = "+55" + "".join(splited)
    elif len(splited) == 3:
        if len(splited[0]) > 2:
            out = False
        elif len(splited[1]) not in [2, 3]:
            out = False
        elif len(splited[2]) not in [8, 9]:
            out = False
        else:
            splited[0] = splited[0].zfill(2)
            splited[1] = splited[1].zfill(3)
            splited[2] = splited[2].zfill(9)
            out = "".join(splited)
    elif len(splited) == 1:
        if len(splited[0]) in [10, 11, 12]:
            out = "+55" + splited[0]
        elif len(splited[0]) in [12, 13, 14]:
            out = splited[0]
    else:
        out = False
    if out:
        return out
    else:
        return False

In [ ]:
import pandas as pd

pedidoDF = pd.read_excel("userData\\ariete.xlsx", index_col=0)
pedidoDF["Total do Pedido"] = pd.to_numeric(
    pedidoDF["Total do Pedido"], errors="coerce"
)
pedidoDF["Valor Pago"] = pd.to_numeric(pedidoDF["Valor Pago"], errors="coerce")
pedidoDF = (
    pedidoDF.groupby("Nome")
    .agg(
        {"Total do Pedido": "sum", "Valor Pago": "sum", "Número para contato": "first"}
    )
    .reset_index()
)

In [ ]:
filtroDF = pedidoDF.query("`Total do Pedido` > `Valor Pago`")
filtroDF["Restante"] = filtroDF["Total do Pedido"] - filtroDF["Valor Pago"]
filtroDF["Número para contato"] = filtroDF["Número para contato"].apply(whatsAppContact)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

driver = webdriver.Chrome(options=Options().add_argument("--incognito"))
driver.get("https://web.whatsapp.com/")
driver.implicitly_wait(0.5)
try:
    while 1:
        driver.find_element(value="landing-main", by=By.CLASS_NAME)
        driver.implicitly_wait(1)
except:
    print("Login OK")

In [ ]:
import urllib.parse
import time


for index, row in filtroDF.iterrows():
    nome = row["Nome"].split(" ")[0]
    numero = row["Número para contato"]
    valor = row["Restante"]
    msg = f"""Bom dia, {nome}. Tudo bem ?"""
    driver.get(
        f"https://web.whatsapp.com/send?phone={numero}&text="
        + urllib.parse.quote_plus(msg)
        + "&type=phone_number&app_absent=1"
    )
    time.sleep(30)
    driver.find_element(By.XPATH, '//button[@aria-label="Enviar"]').click()
    time.sleep(10)
    print(nome)

In [ ]:
driver.find_elements_by_xpath(
    "//*[contains(text(), 'O número de telefone compartilhado por url é inválido.')]"
)